##### Copyright 2020 The OpenFermion Developers

# FQE vs OpenFermion vs Cirq: Diagonal Coulomb Operators

Special Routines are available for evolving under a diagonal Coulomb operator.  This notebook describes how to use these built in routines and how they work.

In [1]:
from itertools import product
import fqe
from fqe.hamiltonians.diagonal_coulomb import DiagonalCoulomb

import numpy as np

import openfermion as of

from scipy.linalg import expm

In [2]:
#Utility function
def uncompress_tei(tei_mat, notation='chemistry'):
    """
    uncompress chemist notation integrals

    tei_tensor[i, k, j, l] = tei_mat[(i, j), (k, l)]
    [1, 1, 2, 2] = [1, 1, 2, 2] = [1, 1, 2, 2]  = [1, 1, 2, 2]
    [i, j, k, l] = [k, l, i, j] = [j, i, l, k]* = [l, k, j, i]*

    For real we also have swap of i <> j and k <> l
    [j, i, k, l] = [l, k, i, j] = [i, j, l, k] = [k, l, j, i]

    tei_mat[(i, j), (k, l)] = int dr1 int dr2 phi_i(dr1) phi_j(dr1) O(r12) phi_k(dr1) phi_l(dr1)

    Physics notation is the notation that is used in FQE.

    Args:
        tei_mat: compressed two electron integral matrix

    Returns:
        uncompressed 4-electron integral tensor. No antisymmetry.
    """
    if notation not in ['chemistry', 'physics']:
        return ValueError("notation can be [chemistry, physics]")

    norbs = int(0.5 * (np.sqrt(8 * tei_mat.shape[0] + 1) - 1))
    basis = {}
    cnt = 0
    for i, j in product(range(norbs), repeat=2):
        if i >= j:
            basis[(i, j)] = cnt
            cnt += 1

    tei_tensor = np.zeros((norbs, norbs, norbs, norbs))
    for i, j, k, l in product(range(norbs), repeat=4):
        if i >= j and k >= l:
            tei_tensor[i, j, k, l] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[k, l, i, j] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[j, i, l, k] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[l, k, j, i] = tei_mat[basis[(i, j)], basis[(k, l)]]

            tei_tensor[j, i, k, l] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[l, k, i, j] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[i, j, l, k] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[k, l, j, i] = tei_mat[basis[(i, j)], basis[(k, l)]]

    if notation == 'chemistry':
        return tei_tensor
    elif notation == 'physics':
        return np.asarray(tei_tensor.transpose(0, 2, 1, 3), order='C')

    return tei_tensor


The first example we will perform is diagonal Coulomb evolution on the Hartree-Fock state.  The diagonal Coulomb operator is defined as

\begin{align}
V = \sum_{\alpha, \beta \in \{\uparrow, \downarrow\}}\sum_{p,q} V_{pq,pq}n_{p,\alpha}n_{q,\beta}
\end{align}

The number of free parpameters are $\mathcal{O}(N^{2})$ where $N$ is the rank of the spatial basis. The `DiagonalCoulomb` Hamiltonian takes either a generic 4-index tensor or the $N \times N$ matrix defining $V$.  If the 4-index tensor is given the $N \times N$ matrix is constructed along with the diagonal correction.  If the goal is to just evolve under $V$ it is recommended the user input the $N \times N$ matrix directly.

All the terms in $V$ commute and thus we can evolve under $V$ exactly by counting the accumulated phase on each bitstring.


To start out let's define a Hartree-Fock wavefunction for 4-orbitals and 2-electrons $S_{z} =0$.

In [3]:
norbs = 4
tedim = norbs * (norbs + 1) // 2
if (norbs // 2) % 2 == 0:
    n_elec = norbs // 2
else:
    n_elec = (norbs // 2) + 1
sz = 0
fqe_wfn = fqe.Wavefunction([[n_elec, sz, norbs]])
fci_data = fqe_wfn.sector((n_elec, sz))
fci_graph = fci_data.get_fcigraph()
hf_wf = np.zeros((fci_data.lena(), fci_data.lenb()), dtype=np.complex128)
hf_wf[0, 0] = 1  # right most bit is zero orbital.
fqe_wfn.set_wfn(strategy='from_data',
                raw_data={(n_elec, sz): hf_wf})
fqe_wfn.print_wfn()

Sector N = 2 : S_z = 0
a'0001'b'0001' (1+0j)


Now we can define a random 2-electron operator $V$.  To define $V$ we need a $4 \times 4$ matrix.  We will generate this matrix by making a full random two-electron integral matrix and then just take the diagonal elements

In [4]:
tei_compressed = np.random.randn(tedim**2).reshape((tedim, tedim))
tei_compressed = 0.5 * (tei_compressed + tei_compressed.T)
tei_tensor = uncompress_tei(tei_compressed, notation='physics')

diagonal_coulomb = of.FermionOperator()
diagonal_coulomb_mat = np.zeros((norbs, norbs))
for i, j in product(range(norbs), repeat=2):
    diagonal_coulomb_mat[i, j] = tei_tensor[i, j, i, j]
    for sigma, tau in product(range(2), repeat=2):
        diagonal_coulomb += of.FermionOperator(
            ((2 * i + sigma, 1), (2 * i + sigma, 0), (2 * j + tau, 1),
             (2 * j + tau, 0)), coefficient=diagonal_coulomb_mat[i, j])

dc_ham = DiagonalCoulomb(diagonal_coulomb_mat)


Evolution under $V$ can be computed by looking at each bitstring, seeing if $n_{p\alpha}n_{q\beta}$ is non-zero and then phasing that string by $V_{pq}$.  For the Hartree-Fock state we can easily calculate this phase accumulation.  The alpha and beta bitstrings are "0001" and "0001". 

In [5]:
alpha_occs = [list(range(fci_graph.nalpha()))]
beta_occs = [list(range(fci_graph.nbeta()))]
occs = alpha_occs[0] + beta_occs[0]
diag_ele = 0.
for ind in occs:
    for jnd in occs:
        diag_ele += diagonal_coulomb_mat[ind, jnd]
evolved_phase = np.exp(-1j * diag_ele)
print(evolved_phase)

# evolve FQE wavefunction
evolved_hf_wfn = fqe_wfn.time_evolve(1, dc_ham)

# check they the accumulated phase is equivalent!
assert np.isclose(evolved_hf_wfn.get_coeff((n_elec, sz))[0, 0], evolved_phase)


(-0.7477186016071566-0.6640157323517555j)


We can now try this out for more than 2 electrons.  Let's reinitialize a wavefunction on 6-orbitals with 4-electrons $S_{z} = 0$ to a random state.

In [6]:
norbs = 6
tedim = norbs * (norbs + 1) // 2
if (norbs // 2) % 2 == 0:
    n_elec = norbs // 2
else:
    n_elec = (norbs // 2) + 1
sz = 0
fqe_wfn = fqe.Wavefunction([[n_elec, sz, norbs]])
fqe_wfn.set_wfn(strategy='random')
inital_coeffs = fqe_wfn.get_coeff((n_elec, sz)).copy()
print("Random initial wavefunction")
fqe_wfn.print_wfn()

Random initial wavefunction
Sector N = 4 : S_z = 0
a'000011'b'000011' (0.056561869141931086-0.032217975234018204j)
a'000011'b'000101' (-0.011861682157371897+0.06410928331162095j)
a'000011'b'001001' (0.012226410258388859+0.04102401743807688j)
a'000011'b'010001' (0.024537418508521164-0.030619325280584385j)
a'000011'b'100001' (-0.07057024874462473+0.00647501879782207j)
a'000011'b'000110' (-0.02579636966698562+0.03748354254259163j)
a'000011'b'001010' (-0.040319999140943065+0.0325298309905296j)
a'000011'b'010010' (-0.010306633008763746-0.08078102692628458j)
a'000011'b'100010' (-0.11274259241461648+0.04064221445498664j)
a'000011'b'001100' (0.0017320839681181056+0.04069004183324782j)
a'000011'b'010100' (-0.058048776799182175-0.02708805537665024j)
a'000011'b'100100' (0.023832975988556927+0.006588623966441798j)
a'000011'b'011000' (0.010303962932304142+0.023620122914510214j)
a'000011'b'101000' (-0.014278041026395273-0.06342696214046654j)
a'000011'b'110000' (0.035285634742391776-0.044672613447469

We need to build our Diagoanl Coulomb operator For this bigger system.

In [7]:
tei_compressed = np.random.randn(tedim**2).reshape((tedim, tedim))
tei_compressed = 0.5 * (tei_compressed + tei_compressed.T)
tei_tensor = uncompress_tei(tei_compressed, notation='physics')

diagonal_coulomb = of.FermionOperator()
diagonal_coulomb_mat = np.zeros((norbs, norbs))
for i, j in product(range(norbs), repeat=2):
    diagonal_coulomb_mat[i, j] = tei_tensor[i, j, i, j]
    for sigma, tau in product(range(2), repeat=2):
        diagonal_coulomb += of.FermionOperator(
            ((2 * i + sigma, 1), (2 * i + sigma, 0), (2 * j + tau, 1),
             (2 * j + tau, 0)), coefficient=diagonal_coulomb_mat[i, j])

dc_ham = DiagonalCoulomb(diagonal_coulomb_mat)


Now we can convert our wavefunction to a cirq wavefunction, evolve under the diagonal_coulomb operator we constructed and then compare the outputs.

In [8]:
cirq_wfn = fqe.to_cirq(fqe_wfn).reshape((-1, 1))
final_cirq_wfn = expm(-1j * of.get_sparse_operator(diagonal_coulomb)) @ cirq_wfn
# recover a fqe wavefunction
from_cirq_wfn = fqe.from_cirq(final_cirq_wfn.flatten(), 1.0E-8)


/Users/nickrubin/anaconda3/envs/fqe-dev/lib/python3.6/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [9]:
fqe_wfn = fqe_wfn.time_evolve(1, dc_ham)
print("Evolved wavefunction")
fqe_wfn.print_wfn()

Evolved wavefunction
Sector N = 4 : S_z = 0
a'000011'b'000011' (0.027053971205271615-0.05920579035054832j)
a'000011'b'000101' (0.06459268452850918+0.00885916563411512j)
a'000011'b'001001' (-0.039305527664165923+0.016956727561872314j)
a'000011'b'010001' (-0.0075746161992381565-0.038500041261468665j)
a'000011'b'100001' (0.0668786526078278+0.023437826300061314j)
a'000011'b'000110' (-0.045408949246946564-0.002914786068699323j)
a'000011'b'001010' (-0.03886826488506601+0.034251280557410016j)
a'000011'b'010010' (-0.042880651584369184-0.06923186199245539j)
a'000011'b'100010' (0.11100187604523444-0.04518036359539257j)
a'000011'b'001100' (0.03720651220811483+0.01656366712329948j)
a'000011'b'010100' (0.008444796845192337-0.06349888690530858j)
a'000011'b'100100' (0.018781592674951317+0.016083298381691404j)
a'000011'b'011000' (0.007020540676141707-0.02479503714902456j)
a'000011'b'101000' (-0.00866352584791332+0.0644343487730119j)
a'000011'b'110000' (-0.044121525250702794+0.03597234244460737j)
a'000

In [10]:
print("From Cirq Evolution")
from_cirq_wfn.print_wfn()
assert np.allclose(from_cirq_wfn.get_coeff((n_elec, sz)),
                   fqe_wfn.get_coeff((n_elec, sz)))
print("Wavefunctions are equivalent")

From Cirq Evolution
Sector N = 4 : S_z = 0
a'000011'b'000011' (0.02705397120527181-0.05920579035054821j)
a'000011'b'000101' (0.06459268452850916+0.008859165634115207j)
a'000011'b'001001' (-0.03930552766416594+0.01695672756187231j)
a'000011'b'010001' (-0.00757461619923808-0.03850004126146867j)
a'000011'b'100001' (0.06687865260782776+0.023437826300061422j)
a'000011'b'000110' (-0.04540894924694657-0.002914786068699324j)
a'000011'b'001010' (-0.03886826488506602+0.03425128055741j)
a'000011'b'010010' (-0.04288065158436916-0.06923186199245536j)
a'000011'b'100010' (0.11100187604523443-0.04518036359539256j)
a'000011'b'001100' (0.03720651220811485+0.01656366712329945j)
a'000011'b'010100' (0.00844479684519234-0.0634988869053086j)
a'000011'b'100100' (0.018781592674951327+0.0160832983816914j)
a'000011'b'011000' (0.007020540676141705-0.024795037149024552j)
a'000011'b'101000' (-0.008663525847913352+0.06443434877301189j)
a'000011'b'110000' (-0.044121525250702794+0.035972342444607355j)
a'000101'b'00001

Finally, we can compare against evolving each term of $V$ individually.

In [11]:
fqe_wfn = fqe.Wavefunction([[n_elec, sz, norbs]])
fqe_wfn.set_wfn(strategy='from_data',
                raw_data={(n_elec, sz): inital_coeffs})
for term, coeff in diagonal_coulomb.terms.items():
    op = of.FermionOperator(term, coefficient=coeff)
    fqe_wfn = fqe_wfn.time_evolve(1, op)

assert np.allclose(from_cirq_wfn.get_coeff((n_elec, sz)),
               fqe_wfn.get_coeff((n_elec, sz)))
print("Individual term evolution is equivalent")

Individual term evolution is equivalent
